In [1]:
%reload_ext dotenv
%dotenv -o

In [2]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.tools import DuckDuckGoSearchRun
from langchain.globals import set_debug
from langchain.agents import tool
from langchain.agents import AgentExecutor
import os

In [3]:
from dotenv import load_dotenv
# Load environment variables: OpenAI API key
load_dotenv()

True

In [4]:
openai_api_key = os.environ["OPENAI_API_KEY"]

### Create tool for OpenAI chat

In [5]:
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

In [6]:
# try out the tool
get_word_length.invoke("abc")

3

In [7]:
tools = [get_word_length]

### Create OpenAI chat agent

In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0,
               openai_api_key = openai_api_key)

In [9]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are very powerful assistant, but bad at calculating lengths of words."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [10]:
llm_with_tools = llm.bind_tools(tools)

In [11]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [12]:
from langchain.agents import AgentExecutor
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_executor_m = AgentExecutor(agent=agent, tools=tools, memory = memory, verbose=True)

In [13]:
memory.clear() # In case I am rerunning this cell
chat_history = []
input1 = "how many letters in the word educe?"
result = agent_executor_m.invoke({"input": input1})
agent_executor_m.invoke({"input": "what did I ask in the previous question?"})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educe'}`


5The word "educe" has 5 letters.

> Finished chain.


> Entering new AgentExecutor chain...
I'm sorry, but I don't have access to the previous questions asked. How can I assist you today?

> Finished chain.


{'input': 'what did I ask in the previous question?',
 'chat_history': [HumanMessage(content='how many letters in the word educe?'),
  AIMessage(content='The word "educe" has 5 letters.'),
  HumanMessage(content='what did I ask in the previous question?'),
  AIMessage(content="I'm sorry, but I don't have access to the previous questions asked. How can I assist you today?")],
 'output': "I'm sorry, but I don't have access to the previous questions asked. How can I assist you today?"}

In [14]:
memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='how many letters in the word educe?'), AIMessage(content='The word "educe" has 5 letters.'), HumanMessage(content='what did I ask in the previous question?'), AIMessage(content="I'm sorry, but I don't have access to the previous questions asked. How can I assist you today?")]), return_messages=True, memory_key='chat_history')

In [15]:
chat_history

[]